# Recommendations on starting a new business in the City of London

### Table of contents
-   Introduction
-	Data
-	Methodology
-	Analysis
-	Results
-	Discussion
-	Conclusion


### Introduction: Business Problem
London, the capital and financial hub of England has always been one of the most popular expat destinations in the world. Some of the factors that make London a great place to live in include its living standards, rich culture, diversity, benefits, education, career prospects and healthcare, to name a few. There is plenty to see and do for people of all ages and backgrounds but there is a lot more to this unique city than its attractions. Entrepreneurs and budding business people from around the world head to the Big Smoke when in the process of establishing themselves. London has now also become one of the leading hubs for start-ups in Europe.

In this notebook, we will be exploring the different neighbourhoods of London using Foursquare location data to get reviews, ratings, tips and counts, which would give us an insight into the different businesses within London such as bars, restaurants, grocery stores, hotels etc. Once we have enough data available on different businesses, we can then decide what kind of business to establish and where we would make the most returns on investment. The recommendations will be based on the results returned by Foursquare API on major businesses.


### Data
The recommendations will be based on the data returned by four square such as business name, category, review and location. This will help us understand what kind of business will have a great impact and in which location within London.

Following data sources will be needed to extract/generate the required information:
- **Google Maps API using Geocoding** to get the coordinates/latitudes and longitudes of London.
- **Four Square API** to get the name, address, category, street, latitude and longitude of different businesses.
- We will use pandas json_normalize to transform our json data into a dataframe.
- **Folium Maps** to generate maps for the businesses in London and which areas there are located.


##### Importing all the the required libraries

In [1]:

import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.5 MB/s  eta 0:00:01
Libraries imported.


### Methodology

In this project we will be focusing on manipulating data in the categories of hotels, bars, restaurants and shops within London. We will be analyzing bisinesses that are in 100000 metres radius from London. This will help us get all of the places that have been rated by Foursquare users within and around London.

We will first make an api request to Foursqaure and return our data. The dataset will include hotels, bars, restaurants etc. that have been shared and rated. Our dataset attributes will consist of the place name, address, category, street, latitude and longitude. Our dataset will be in json format when first returned from Foursquare.

Secondly, we will use pandas json_normalize to transform our json data into a dataframe. We will pass the appropriate columns when processing.

Finally, we will replace the appropriate address names to businesses returning NaN. Once our data has been processed properly, we will use Folium to map the businesses using their latitude and logitude values. This will give us a visibility of where these businesses are located within London. We will also be exploring the highest rated business and the kinds of businesses that are situated around it.

Our recommendation will be based on what category of businesses people prefer to visit when they come to London. Breaking it down by category and giving the count will help potential business owners on what category of businesses to make their investment.

#### Defining the Four square credentials

In [2]:
CLIENT_ID = 'YMUTBH2EYCOOPHAOQ4GRW4Q2ROJ1Z2JE0QEURTXC024EUPSI' # your Foursquare ID
CLIENT_SECRET = '5CBANNECNBL5CI4MIKG2U0L1MKZLIQ2P55F1ZHT2HIMFMYNY' # your Foursquare Secret
ACCESS_TOKEN = 'JUIHLCIF3BMUXH1J3FZHZDIX0Z3DBRRSKYQHHJKOIWRRMKY3' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 3000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YMUTBH2EYCOOPHAOQ4GRW4Q2ROJ1Z2JE0QEURTXC024EUPSI
CLIENT_SECRET:5CBANNECNBL5CI4MIKG2U0L1MKZLIQ2P55F1ZHT2HIMFMYNY


### Analysis


Converting London to its latitude and longitude
We will use this to pass in our lat and lng to the api request.

In [3]:

address = 'London, Great Britain'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitudes and longitudes of London are",latitude, longitude)

The latitudes and longitudes of London are 51.5073219 -0.1276474


#### Making an API call using four square

In [4]:
LIMIT = 3000  #setting a linit of 3000
radius = 100000  #within a radius of 100000 metres of London

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YMUTBH2EYCOOPHAOQ4GRW4Q2ROJ1Z2JE0QEURTXC024EUPSI&client_secret=5CBANNECNBL5CI4MIKG2U0L1MKZLIQ2P55F1ZHT2HIMFMYNY&v=20180604&ll=51.5073219,-0.1276474&radius=100000'

#### We will pass the result to json object

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606aeacae401dd6f70a1fead'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'London',
  'headerFullLocation': 'London',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 52.4073228000009,
    'lng': 1.315635686428644},
   'sw': {'lat': 50.607320999999104, 'lng': -1.5709304864286442}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac518cdf964a520f2a520e3',
       'name': "St James's Park",
       'location': {'address': 'The Mall',
        'crossStreet': 'Horse Guards Rd',
        'lat': 51.50325316049429,
        'lng': -0.1329946517944336,
        'labeledLatLngs': [{'label': 'display',
        


We extract the categories of the businesses. This will help us to understand which business category has the highest investment and attraction in London

In [6]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


We convert the json result into a pandas dataframe for manipulation. The dataframe here shows the most visited businesses within London. I also want to mention that this data is mainly supplied by expats who visit these places and give reviews.

In [7]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
places = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.crossStreet', 'venue.location.lat', 'venue.location.lng']
places =places.loc[:, filtered_columns]

# filter the category for each row
places['venue.categories'] = places.apply(get_category_type, axis=1)

# clean columns
places.columns = [col.split(".")[-1] for col in places.columns]

places

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,crossStreet,lat,lng
0,St James's Park,Park,The Mall,Horse Guards Rd,51.503253,-0.132995
1,National Gallery,Art Museum,Trafalgar Sq,NaN,51.508876,-0.128478
2,Somerset House,Event Space,Strand,NaN,51.510786,-0.117899
3,Phillips,Art Gallery,30 Berkeley Sq,NaN,51.510381,-0.147017
4,National Theatre,Theater,Upper Ground,NaN,51.507376,-0.114793
5,Royal Opera House,Opera House,Bow St,Floral St,51.513083,-0.121968
6,The Connaught,Hotel,16 Carlos Pl,NaN,51.510138,-0.149498
7,Trafalgar Square,Plaza,Trafalgar Sq,NaN,51.507987,-0.128048
8,Carpo Piccadilly,Gourmet Shop,16 Piccadilly,NaN,51.509564,-0.135536
9,The London Edition (The London EDITION),Hotel,10 Berners St,NaN,51.516762,-0.136049



In the cross street column, we see that we have NaN for some of the businesses. We will replace them with Unknown to keep it simple.The inportant thing is that we don't have any NaN values in the address column.

In [8]:
places["crossStreet"].fillna("Unknown", inplace = True) 
places.head()

,name,categories,address,crossStreet,lat,lng
0,St James's Park,Park,The Mall,Horse Guards Rd,51.503253,-0.132995
1,National Gallery,Art Museum,Trafalgar Sq,Unknown,51.508876,-0.128478
2,Somerset House,Event Space,Strand,Unknown,51.510786,-0.117899
3,Phillips,Art Gallery,30 Berkeley Sq,Unknown,51.510381,-0.147017
4,National Theatre,Theater,Upper Ground,Unknown,51.507376,-0.114793


Here we use folium library to map all the businesses we have retrieved from Foursquare. We can clearly see the blue marker on places that have businesses in around London.

In [9]:
# create map of London using latitude and longitude values
London_business = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(places['lat'], places['lng'], places['address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(London_business)  
    
London_business

Based on the data returned from Foursquare, we see that St James's Park has the highest ratings. This is so because Foursquare presents data based on the highest to the lowest ratings. We will manipulate other businesses around St James's park by passing its latitude and longitude to the api request. This will let the potential business owners have an idea of the different types of businesses that are situated around St James's Park.

In [10]:
places.loc[0, 'name']

"St James's Park"

In [11]:
StJames_latitude = places.loc[0, 'lat'] # neighborhood latitude value
StJames_longitude = places.loc[0, 'lng'] # neighborhood longitude value

StJames_park = places.loc[0, 'name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(StJames_park, 
                                                               StJames_latitude, 
                                                               StJames_longitude))

Latitude and longitude values of St James's Park are 51.50325316049429, -0.1329946517944336.


In [12]:
LIMIT = 5000
radius = 100000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    StJames_latitude, 
    StJames_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YMUTBH2EYCOOPHAOQ4GRW4Q2ROJ1Z2JE0QEURTXC024EUPSI&client_secret=5CBANNECNBL5CI4MIKG2U0L1MKZLIQ2P55F1ZHT2HIMFMYNY&v=20180604&ll=51.50325316049429,-0.1329946517944336&radius=100000&limit=5000'

In [13]:
near_StJames_Park = requests.get(url).json()
near_StJames_Park

{'meta': {'code': 200, 'requestId': '606aebf5f48cf4549183f333'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'London',
  'headerFullLocation': 'London',
  'headerLocationGranularity': 'city',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 52.403254060495186,
    'lng': 1.3101595662279877},
   'sw': {'lat': 50.60325226049339, 'lng': -1.5761488698168549}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac518cdf964a520f2a520e3',
       'name': "St James's Park",
       'location': {'address': 'The Mall',
        'crossStreet': 'Horse Guards Rd',
        'lat': 51.50325316049429,
        'lng': -0.1329946517944336,
        'labeledLatLngs': [{'label': 'display',
      

Converting json to pandas dataframe

In [14]:
from pandas.io.json import json_normalize

venues = near_StJames_Park['response']['groups'][0]['items']
    
StJames_df = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.crossStreet', 'venue.location.lat', 'venue.location.lng']
StJames_df =StJames_df.loc[:, filtered_columns]

# filter the category for each row
StJames_df['venue.categories'] = StJames_df.apply(get_category_type, axis=1)

# clean columns
StJames_df.columns = [col.split(".")[-1] for col in StJames_df.columns]

StJames_df.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,crossStreet,lat,lng
0,St James's Park,Park,The Mall,Horse Guards Rd,51.503253,-0.132995
1,National Gallery,Art Museum,Trafalgar Sq,NaN,51.508876,-0.128478
2,Phillips,Art Gallery,30 Berkeley Sq,NaN,51.510381,-0.147017
3,Somerset House,Event Space,Strand,NaN,51.510786,-0.117899
4,The Connaught,Hotel,16 Carlos Pl,NaN,51.510138,-0.149498


The dataset below clearly depicts that over 80% of businesses,hotels, art galleries,museums and stores are situated in London. Comparing the first dataset where we used London coordinates with the second one which we used St James's Park with the highest ratings, we can see similar establishments. This suggests that the same group of customers are moving between these businesses. Establishing a business with similar categories in mind, we have an 80% assurance that these same customers will come around

In [15]:
StJames_df

,name,categories,address,crossStreet,lat,lng
0,St James's Park,Park,The Mall,Horse Guards Rd,51.503253,-0.132995
1,National Gallery,Art Museum,Trafalgar Sq,NaN,51.508876,-0.128478
2,Phillips,Art Gallery,30 Berkeley Sq,NaN,51.510381,-0.147017
3,Somerset House,Event Space,Strand,NaN,51.510786,-0.117899
4,The Connaught,Hotel,16 Carlos Pl,NaN,51.510138,-0.149498
5,Mount Street Gardens,Park,Mount St,NaN,51.509143,-0.150203
6,Elizabeth Tower (Big Ben) (Big Ben (Elizabeth ...,Monument / Landmark,Parliament Sq,Bridge St,51.500620,-0.124578
7,Hyde Park,Park,Serpentine Rd,NaN,51.507188,-0.163731
8,Green Park,Park,Piccadilly,Constitution Hill,51.504335,-0.143766
9,Carpo Piccadilly,Gourmet Shop,16 Piccadilly,NaN,51.509564,-0.135536


In [16]:
# create map of London using latitude and longitude values
new_StJames_businesses = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(StJames_df['lat'], StJames_df['lng'], StJames_df['address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(new_StJames_businesses)  
    
new_StJames_businesses

### Results


Grouping the dataset by cagories, we see that the category Park(theme Park and attractions within the park) has the highest amount of entry according to our Foursquare data, followed by hotels. We can advised our new investor to make their investment in the order of Theme Parks, Hotels, Theatre and grocery store. These will have a good return on investments in London based on our dataset.

In [17]:
StJames_df.groupby('categories').count()

,name,address,crossStreet,lat,lng
categories,,,,,
Art Gallery,2,2,1,2,2
Art Museum,2,2,0,2,2
Bakery,2,2,1,2,2
Beer Store,1,1,0,1,1
Brewery,1,1,0,1,1
Café,3,3,0,3,3
Canal,2,1,1,2,2
Castle,1,1,0,1,1
Cheese Shop,1,1,0,1,1


### Exploring Neighborhoods near St James's Park
The function will also repeat the same process to all the neighbourhoods near St James Park
Please note that in this sense, neighbourhoods refer to the name of the businesses and not the physical location.

In [18]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_StJames = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_StJames.columns =  ['Neighbourhood', 
                               'Neighbourhood Latitude', 
                               'Neighbourhood Longitude', 
                               'Venue', 
                               'Venue Latitude', 
                               'Venue Longitude', 
                               'Venue Category']
    
    return(nearby_StJames)

Now we will use the code to run the above function on each neighborhood and create a new dataframe called nearby_StJames_venue.

In [19]:
nearby_StJames_venue = getNearbyVenues(names=StJames_df['name'],
                                       latitudes=StJames_df['lat'],
                                       longitudes=StJames_df['lng']
                                      )

St James's Park
National Gallery
Phillips
Somerset House
The Connaught
Mount Street Gardens
Elizabeth Tower (Big Ben) (Big Ben (Elizabeth Tower))
Hyde Park
Green Park
Carpo Piccadilly
Selfridges Food Hall
Fortnum & Mason
National Theatre
The London Edition (The London EDITION)
Claridge's
Scott's
Trafalgar Square
Royal Opera House
M&S Simply Food
Southbank Centre
British Museum
Rosewood London
Covent Garden Market
The Savoy Hotel
Whitehall Gardens
Hedonism Wines
Borough Market
The London Eye
Selfridges & Co
The Dorchester
Monmouth Coffee Company
Regent's Park
Battersea Park
Royal Albert Hall
Kensington Gardens
Tate Britain
The Hoxton Holborn
Carpo
Connaught Bar
Queen Mary's Gardens
Duke of York Square
Marriott Grosvenor House Executive Lounge
BVLGARI Hotel London
M&S Foodhall
Neal's Yard Dairy
Venchi
Great Bakery & Deli
The Wallace Collection
Zuma
Shakespeare's Globe Theatre
Primrose Hill
Rosslyn
Maltby Street Market
Mercato Metropolitano
OPSO
Dishoom
Barbican Centre
Granary Square
The 

### We can now check the size of the resulting dataframe

In [20]:

nearby_StJames_venue.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St James's Park,51.503253,-0.132995,St James's Park,51.503253,-0.132995,Park
1,St James's Park,51.503253,-0.132995,Duck Island Cottage,51.502834,-0.129868,Garden
2,St James's Park,51.503253,-0.132995,Churchill War Rooms (Churchill Museum & Cabine...,51.502079,-0.129305,Historic Site
3,St James's Park,51.503253,-0.132995,St James's Palace,51.504632,-0.136888,Palace
4,St James's Park,51.503253,-0.132995,St James's Café,51.503998,-0.131390,Café


We can check how many venues were returned for each neighborhood

In [21]:
nearby_StJames_venue.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
1Rebel,100,100,100,100,100,100
Amoret Speciality Coffee,77,77,77,77,77,77
BVLGARI Hotel London,79,79,79,79,79,79
Barbican Centre,45,45,45,45,45,45
Battersea Park,13,13,13,13,13,13
Borough Market,100,100,100,100,100,100
Brawn,46,46,46,46,46,46
Brew by Numbers,42,42,42,42,42,42
Brickwood Coffee & Bread,77,77,77,77,77,77


### Unique categories from all returned venues



In [22]:

print('There are {} uniques categories.'.format(len(nearby_StJames_venue['Venue Category'].unique())))

There are 319 uniques categories.


### Extra Analysis of each Neighbourhood around St JAmes's Park

In [23]:
# one hot encoding
nearby_StJames_onehot = pd.get_dummies(nearby_StJames_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nearby_StJames_onehot['Neighbourhood'] = nearby_StJames_venue['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nearby_StJames_onehot.columns[-1]] + list(nearby_StJames_onehot.columns[:-1])
nearby_StJames_onehot = nearby_StJames_onehot[fixed_columns]

nearby_StJames_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stables,Stationery Sto

In [46]:
nearby_StJames_onehot.shape

(6503, 322)


Next, we will group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:

nearby_StJames_grouped = nearby_StJames_onehot.groupby('Neighbourhood').mean().reset_index()
nearby_StJames_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bathing Area,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shaanxi Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stables,Stationery Sto

In [25]:
nearby_StJames_grouped.shape

(99, 320)

Let's print each neighborhood along with the top 5 most common venues

In [26]:
num_top_venues = 5

for hood in nearby_StJames_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = nearby_StJames_grouped[nearby_StJames_grouped['Neighbourhood'] == hood].T.reset_index()
    temp = nearby_StJames_grouped[nearby_StJames_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1Rebel----
          venue  freq
0   Coffee Shop  0.08
1         Hotel  0.05
2   Pizza Place  0.04
3  Cocktail Bar  0.04
4    Restaurant  0.03


----Amoret Speciality Coffee----
                venue  freq
0                 Pub  0.09
1  Italian Restaurant  0.05
2         Coffee Shop  0.05
3                Café  0.04
4  Persian Restaurant  0.04


----BVLGARI Hotel London----
                venue  freq
0                Café  0.09
1  Italian Restaurant  0.08
2            Boutique  0.06
3         Coffee Shop  0.05
4               Hotel  0.05


----Barbican Centre----
                  venue  freq
0           Coffee Shop  0.09
1  Gym / Fitness Center  0.09
2            Food Truck  0.07
3   Indie Movie Theater  0.04
4                 Plaza  0.04


----Battersea Park----
          venue  freq
0          Café  0.23
1        Garden  0.08
2   Art Gallery  0.08
3  Soccer Field  0.08
4          Park  0.08


----Borough Market----
                venue  freq
0         Coffee Shop  0.12
1      

### We will put the reults in pandas dataframe

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top ten venues for each Neighbourhood

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = nearby_StJames_grouped['Neighbourhood']

for ind in np.arange(nearby_StJames_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nearby_StJames_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1Rebel,Coffee Shop,Hotel,Cocktail Bar,Pizza Place,Restaurant,Mediterranean Restaurant,Salad Place,Wine Bar,Middle Eastern Restaurant,Indian Restaurant
1,Amoret Speciality Coffee,Pub,Coffee Shop,Italian Restaurant,Persian Restaurant,Indian Restaurant,Café,Ice Cream Shop,Restaurant,Park,Boutique
2,BVLGARI Hotel London,Café,Italian Restaurant,Boutique,Hotel,Coffee Shop,Pizza Place,Bakery,Indian Restaurant,Jewelry Store,Grocery Store
3,Barbican Centre,Coffee Shop,Gym / Fitness Center,Food Truck,Indie Movie Theater,Plaza,Food Stand,Gastropub,Bike Shop,Garden,Sushi Restaurant
4,Battersea Park,Café,Park,Track Stadium,Zoo,Plaza,Playground,Soccer Field,Art Gallery,Monument / Landmark,Tennis Court


### Conclusion
In this notebook, we identified popular businesses and business categories in London in order to aid potential business owners on what type of businesses to venture with. We acquired our data from Foursquare and processed it, which included the business names, categories, address, street, latitude and longitude. We further mapped the businesses using Folium.

After exploring the top venues (categories), we see that businesses in the categories of coffee shops,Pubs,Cafe,Hotel restaurants are the top most attractive businesses situated in London. We can conclude that venturing in any of these categories will yield returns in investment